Настройка окружения

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz


In [3]:
!tar xvzf spark-3.2.1-bin-hadoop3.2.tgz > /dev/null

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [5]:
!wget https://repo1.maven.org/maven2/org/apache/spark/spark-avro_2.12/3.2.1/spark-avro_2.12-3.2.1.jar  -P $SPARK_HOME/jars/

!echo spark.executor.extraClassPath $SPARK_HOME/jars/spark-avro_2.12/3.2.1.jar

!echo spark.driver.extraClassPath $SPARK_HOME/jars/spark-avro_2.12/3.2.1.jar

--2024-01-19 07:46:41--  https://repo1.maven.org/maven2/org/apache/spark/spark-avro_2.12/3.2.1/spark-avro_2.12-3.2.1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186483 (182K) [application/java-archive]
Saving to: ‘/content/spark-3.2.1-bin-hadoop3.2/jars/spark-avro_2.12-3.2.1.jar.9’

spark-avro_2.12-3.2 100%[===================>] 182.11K  --.-KB/s    in 0.03s   

2024-01-19 07:46:41 (5.48 MB/s) - ‘/content/spark-3.2.1-bin-hadoop3.2/jars/spark-avro_2.12-3.2.1.jar.9’ saved [186483/186483]

spark.executor.extraClassPath /content/spark-3.2.1-bin-hadoop3.2/jars/spark-avro_2.12/3.2.1.jar
spark.driver.extraClassPath /content/spark-3.2.1-bin-hadoop3.2/jars/spark-avro_2.12/3.2.1.jar


In [6]:
!pip install pyspark==3.2.1

In [7]:
!pip install findspark

In [8]:
!echo $SPARK_HOME

/content/spark-3.2.1-bin-hadoop3.2


In [9]:
import findspark
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import  SparkContext
from pyspark.sql import functions as f
appName = "PySpark Task 1"
#master = "spark://10.3.100.4:7077"
master = 'local[*]'
# Create Spark session with Hive supported.
spark = SparkSession.builder \
    .enableHiveSupport() \
    .appName(appName) \
    .getOrCreate()

Чтение исходных данных

In [10]:
PATH = 'data_for_exam/countries'
countries = spark.read.orc(PATH)

countries = spark.read.orc(PATH)
countries.printSchema()
countries.show(5)

root
 |-- COUNTRY_ID: string (nullable = true)
 |-- COUNTRY_NAME: string (nullable = true)
 |-- REGION_ID: integer (nullable = true)

+----------+------------+---------+
|COUNTRY_ID|COUNTRY_NAME|REGION_ID|
+----------+------------+---------+
|        AR|   Argentina|        2|
|        AU|   Australia|        3|
|        BE|     Belgium|        1|
|        BR|      Brazil|        2|
|        CA|      Canada|        2|
+----------+------------+---------+
only showing top 5 rows



In [11]:
PATH = 'data_for_exam/departments/departments'

departments = spark.read\
.option('sep', ',')\
.option('header', 'True')\
.option('inferSchema', True)\
.csv(PATH)

departments.printSchema()
print()
departments.show(5)

root
 |-- DEPARTMENT_ID: integer (nullable = true)
 |-- DEPARTMENT_NAME: string (nullable = true)
 |-- MANAGER_ID: integer (nullable = true)
 |-- LOCATION_ID: integer (nullable = true)


+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           10| Administration|       200|       1700|
|           20|      Marketing|       201|       1800|
|           30|     Purchasing|       114|       1700|
|           40|Human Resources|       203|       2400|
|           50|       Shipping|       121|       1500|
+-------------+---------------+----------+-----------+
only showing top 5 rows



In [12]:
PATH = 'data_for_exam/employees/employees'

employees = spark.read\
.option('sep', '\t')\
.option('header', 'True')\
.option('inferSchema', True)\
.csv(PATH)

employees.printSchema()
print()
employees.show(5)

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: integer (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)


+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567| 17.06.03|AD_PRES| 24000|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|5

In [13]:
PATH = 'data_for_exam/job_history/job_history'

job_history = spark.read.json(PATH)

job_history.printSchema()
print()
job_history.show(5)

root
 |-- department_id: long (nullable = true)
 |-- employee_id: long (nullable = true)
 |-- end_date: string (nullable = true)
 |-- job_id: string (nullable = true)
 |-- start_date: string (nullable = true)


+-------------+-----------+--------+----------+----------+
|department_id|employee_id|end_date|    job_id|start_date|
+-------------+-----------+--------+----------+----------+
|           60|        102|24.07.06|   IT_PROG|  13.01.01|
|          110|        101|27.10.01|AC_ACCOUNT|  21.09.97|
|          110|        101|15.03.05|    AC_MGR|  28.10.01|
|           20|        201|19.12.07|    MK_REP|  17.02.04|
|           50|        114|31.12.07|  ST_CLERK|  24.03.06|
+-------------+-----------+--------+----------+----------+
only showing top 5 rows



In [14]:
PATH = 'data_for_exam/jobs/jobs'

jobs = spark.read\
.option('sep', ';')\
.option('header', 'True')\
.option('inferSchema', True)\
.csv(PATH)

jobs.printSchema()
print()
jobs.show(5)

root
 |-- JOB_ID: string (nullable = true)
 |-- JOB_TITLE: string (nullable = true)
 |-- MIN_SALARY: integer (nullable = true)
 |-- MAX_SALARY: integer (nullable = true)


+----------+--------------------+----------+----------+
|    JOB_ID|           JOB_TITLE|MIN_SALARY|MAX_SALARY|
+----------+--------------------+----------+----------+
|   AD_PRES|           President|     20080|     40000|
|     AD_VP|Administration Vi...|     15000|     30000|
|   AD_ASST|Administration As...|      3000|      6000|
|    FI_MGR|     Finance Manager|      8200|     16000|
|FI_ACCOUNT|          Accountant|      4200|      9000|
+----------+--------------------+----------+----------+
only showing top 5 rows



In [15]:
PATH = 'data_for_exam/locations'

locations = spark.read.parquet(PATH)
locations.printSchema()
locations.show(5)

root
 |-- LOCATION_ID: integer (nullable = true)
 |-- STREET_ADDRESS: string (nullable = true)
 |-- POSTAL_CODE: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE_PROVINCE: string (nullable = true)
 |-- COUNTRY_ID: string (nullable = true)

+-----------+--------------------+-----------+---------+----------------+----------+
|LOCATION_ID|      STREET_ADDRESS|POSTAL_CODE|     CITY|  STATE_PROVINCE|COUNTRY_ID|
+-----------+--------------------+-----------+---------+----------------+----------+
|       1000|1297 Via Cola di Rie|      00989|     Roma|            null|        IT|
|       1100|93091 Calle della...|      10934|   Venice|            null|        IT|
|       1200|    2017 Shinjuku-ku|       1689|    Tokyo|Tokyo Prefecture|        JP|
|       1300|     9450 Kamiya-cho|       6823|Hiroshima|            null|        JP|
|       1400| 2014 Jabberwocky Rd|      26192|Southlake|           Texas|        US|
+-----------+--------------------+-----------+---------+-

In [16]:
employees.createOrReplaceTempView('employees')
countries.createOrReplaceTempView('countries')
locations.createOrReplaceTempView('locations')
departments.createOrReplaceTempView('departments')

Функции для чтения и записи файлов Spark

In [17]:
def spark_write(df, format, compression, path):
  df.write\
  .format(format)\
  .option('compression', compression)\
  .mode('overwrite')\
  .save(path)

def spark_read(format, compression, path):
  spark.read.format(format)\
  .option('compression', compression)\
  .load(path)\
  .show(5, False)

1. Напишите запрос, возвращающий строку следующего вида для каждого сотрудника: "<фамилия> зарабатывает <зарплата> каждый месяц, но хочет получать <зарплата * 3>". Назовите колонку 'Dream Salaries'. Результат сохранить в формате parquet со сжатием GZIP

In [18]:
QUERY = '''
select last_name || ' зарабатывает ' || salary || ' каждый месяц, но хочет получать '|| salary * 3
as Dream_Salaries
from employees
'''

df = spark.sql(QUERY)

spark_write(df, 'parquet', 'gzip', 'results/task1')
spark_read('parquet', 'gzip', 'results/task1')

+----------------------------------------------------------------+
|Dream_Salaries                                                  |
+----------------------------------------------------------------+
|King зарабатывает 24000 каждый месяц, но хочет получать 72000   |
|Kochhar зарабатывает 17000 каждый месяц, но хочет получать 51000|
|De Haan зарабатывает 17000 каждый месяц, но хочет получать 51000|
|Hunold зарабатывает 9000 каждый месяц, но хочет получать 27000  |
|Ernst зарабатывает 6000 каждый месяц, но хочет получать 18000   |
+----------------------------------------------------------------+
only showing top 5 rows



2. Напишите запрос, возвращающий адреса всех департаментов. Запрос должен возвращать ID локации, адрес (улицу), город, штат и страну.

In [19]:
QUERY = '''
select location_id, street_address, city, state_province, country_name
from departments
natural join locations
natural join countries
order by location_id
'''

df = spark.sql(QUERY)

spark_write(df, 'parquet', 'gzip', 'results/task2')
spark_read('parquet', 'gzip', 'results/task2')

+-----------+-------------------+-------------------+--------------+------------------------+
|location_id|street_address     |city               |state_province|country_name            |
+-----------+-------------------+-------------------+--------------+------------------------+
|1400       |2014 Jabberwocky Rd|Southlake          |Texas         |United States of America|
|1500       |2011 Interiors Blvd|South San Francisco|California    |United States of America|
|1700       |2004 Charade Rd    |Seattle            |Washington    |United States of America|
|1700       |2004 Charade Rd    |Seattle            |Washington    |United States of America|
|1700       |2004 Charade Rd    |Seattle            |Washington    |United States of America|
+-----------+-------------------+-------------------+--------------+------------------------+
only showing top 5 rows



3. Напишите запрос, возвращающий фамилию, ID отдела и наименование отдела для каждого сотрудника; Результат сохранить в формате avro со сжатием GZIP

(прим. avro не поддерживает gzip)

In [20]:
QUERY = '''
select last_name, department_id, department_name
from employees
natural join departments
order by last_name
'''

df = spark.sql(QUERY)

spark_write(df, 'avro', 'deflate', 'results/task3')
spark_read('avro', 'deflate', 'results/task3')

+---------+-------------+---------------+
|last_name|department_id|department_name|
+---------+-------------+---------------+
|Atkinson |50           |Shipping       |
|Austin   |60           |IT             |
|Baida    |30           |Purchasing     |
|Bernstein|80           |Sales          |
|Bissot   |50           |Shipping       |
+---------+-------------+---------------+
only showing top 5 rows



4. Напишите запрос, возвращающий фамилию, ID сотрудника, фамилию менеджера и ID менеджера для каждого сотрудника (для сотрудников, у которых нет менеджера, в этих колонках должен быть NULL).
Назовите колонки 'Employee', 'Emp#', 'Manager', 'Mgr#'. Результат сохранить в формате avro со сжатием Snappy

In [21]:
QUERY = '''
select e.last_name as Employee, e.employee_id as Emp_N, m.last_name as Manager, e.manager_id as Mgr_N
from employees as e
left join employees as m on e.manager_id = m.employee_id
order by e.employee_id
'''

df = spark.sql(QUERY)

spark_write(df, 'avro', 'snappy', 'results/task4')
spark_read('avro', 'snappy', 'results/task4')

+--------+-----+-------+-----+
|Employee|Emp_N|Manager|Mgr_N|
+--------+-----+-------+-----+
|King    |100  |null   |null |
|Kochhar |101  |King   |100  |
|De Haan |102  |King   |100  |
|Hunold  |103  |De Haan|102  |
|Ernst   |104  |Hunold |103  |
+--------+-----+-------+-----+
only showing top 5 rows



5. Напишите запрос, возвращающий фамилии и зарплаты всех сотрудников, которые подчиняются сотруднику King. Используйте подзапрос.

In [22]:
QUERY = '''
select last_name, salary
from employees
where manager_id in (select employee_id
                     from employees
                     where last_name = 'King')
'''

df = spark.sql(QUERY)

spark_write(df, 'parquet', 'gzip', 'results/task5')
spark_read('parquet', 'gzip', 'results/task5')

+---------+------+
|last_name|salary|
+---------+------+
|Kochhar  |17000 |
|De Haan  |17000 |
|Raphaely |11000 |
|Weiss    |8000  |
|Fripp    |8200  |
+---------+------+
only showing top 5 rows



6. Напишите запрос, возвращающий фамилии всех сотрудников, получающих больше, чем любой сотрудник отдела с ID 60.

In [23]:
QUERY = '''
select last_name
from employees
where salary > (select max(salary)
				        from employees
				        where department_id = 60)
'''
df = spark.sql(QUERY)

spark_write(df, 'parquet', 'gzip', 'results/task6')
spark_read('parquet', 'gzip', 'results/task6')

+---------+
|last_name|
+---------+
|King     |
|Kochhar  |
|De Haan  |
|Greenberg|
|Raphaely |
+---------+
only showing top 5 rows



7. Напишите запрос, возвращающий ID, фамилии и зарплаты всех сотрудников, работающих в одном отделе с работником, в чьей фамилии есть буква 'u' и получающих больше средней зарплаты в компании.

In [24]:
QUERY = '''
select employee_id, last_name
from employees
where department_id in (select distinct(department_id)
						            from employees
						            where last_name like '%u%')
and employee_id not in (select employee_id
						            from employees
						            where last_name like '%u%')
and salary > (select avg(salary)
			        from employees)
order by salary
'''

df = spark.sql(QUERY)

spark_write(df, 'parquet', 'gzip', 'results/task7')
spark_read('parquet', 'gzip', 'results/task7')

+-----------+---------+
|employee_id|last_name|
+-----------+---------+
|123        |Vollman  |
|165        |Lee      |
|161        |Sewall   |
|164        |Marvins  |
|172        |Bates    |
+-----------+---------+
only showing top 5 rows



8. Выведите фамилии, id отдела и название отдела для всех сотрудников, не привязанных ни к одному отделу, а также список отделов, к которым не привязан ни один сотрудник.

In [25]:
QUERY = '''
select last_name, employees.department_id, department_name
from employees
left join departments on employees.department_id = departments.department_id
where employees.department_id is null

union

select last_name, employees.department_id, department_name
from departments
left join employees on employees.department_id = departments.department_id
where employees.last_name is null
'''

df = spark.sql(QUERY)

spark_write(df, 'parquet', 'gzip', 'results/task8')
spark_read('parquet', 'gzip', 'results/task8')

+---------+-------------+----------------+
|last_name|department_id|department_name |
+---------+-------------+----------------+
|Grant    |null         |null            |
|null     |null         |Corporate Tax   |
|null     |null         |Government Sales|
|null     |null         |Payroll         |
|null     |null         |Recruiting      |
+---------+-------------+----------------+
only showing top 5 rows

